# Prediction model

We implement the feature engineering done in "analyze_features_compilability" to the prediction model.

In [19]:
import psycopg2
import pandas as pd
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
import re
import numpy as np

In [2]:
sql_nc = '''
select cs.cwhen as breaker
    , cm.cwhen as child
    , cs.message as breaker
    , cm.message as child
    , Substring(cs.csha,0,8) as broken
    , Substring(parent.prev,0,8) as parent 
    , Substring(child.curr,0,8) as child
    , cs.email as breaker
    , cm.email as child
	, cp.cwhen as parentTime
	, cp.message as parentMessage
from
impact_pairs as parent
, impact_pairs as child
, commits as cs
, commits as cm
, commits as cp
,(
    (select prev as csha from impact_pairs) 
    union 
    (select curr as csha from impact_pairs)
) as ip
,(
    select cs.application as app
    , min(cwhen) as minwhen
    , max(cwhen) as maxwhen
    from 
    (
        (select prev as csha from impact_pairs) 
        union 
        (select curr as csha from impact_pairs)
    )
    as ip
    ,commits as cs
    ,(
        (select application,csha from findbugs_summary_uni) 
        union 
        (select application,csha from pmd_uni) 
        union 
        (select application,csha from sonarqube_system_uni) 
    )
    as anal
    where cs.csha=anal.csha
    and cs.csha=ip.csha
    and cs.application=anal.application
    and cs.application like %s
    group by cs.application
) as recovered
where
    cs.application = recovered.app
    and cs.cwhen <= recovered.maxwhen
    and cs.cwhen >= recovered.minwhen
    and cs.csha=ip.csha
    and parent.curr=cs.csha
    and child.prev=cs.csha
    and cm.csha=child.curr
	and cp.application=parent.application
	and cp.csha=parent.prev

    and ip.csha not in
    (
        select cs.csha
        from 
        (
            (select prev as csha from impact_pairs) 
            union 
            (select curr as csha from impact_pairs)
        )
        as ip
        ,commits as cs
        ,(
            (select application,csha from findbugs_summary_uni) 
            union 
            (select application,csha from pmd_uni) 
            union 
            (select application,csha from sonarqube_system_uni) 
        )
        as anal
        where cs.csha=anal.csha
        and cs.csha=ip.csha
        and cs.application=anal.application
    )
order by cs.application asc ,  cs.cwhen desc

'''


sql_c = '''

select cs.cwhen as breaker
    , cm.cwhen as child
    , cs.message as breaker
    , cm.message as child
    , Substring(cs.csha,0,8) as broken
    , Substring(parent.prev,0,8) as parent 
    , Substring(child.curr,0,8) as child
    , cs.email as breaker
    , cm.email as child
	, cp.cwhen as parentTime
	, cp.message as parentMessage
from
impact_pairs as parent
, impact_pairs as child
, commits as cs
, commits as cm
, commits as cp
,(
    (select prev as csha from impact_pairs) 
    union 
    (select curr as csha from impact_pairs)
) as ip
,(
    select cs.application as app
    , min(cwhen) as minwhen
    , max(cwhen) as maxwhen
    from 
    (
        (select prev as csha from impact_pairs) 
        union 
        (select curr as csha from impact_pairs)
    )
    as ip
    ,commits as cs
    ,(
        (select application,csha from findbugs_summary_uni) 
        union 
        (select application,csha from pmd_uni) 
        union 
        (select application,csha from sonarqube_system_uni) 
    )
    as anal
    where cs.csha=anal.csha
    and cs.csha=ip.csha
    and cs.application=anal.application
    and cs.application like %s
    group by cs.application
) as recovered
where
    cs.application = recovered.app
    and cs.cwhen <= recovered.maxwhen
    and cs.cwhen >= recovered.minwhen
    and cs.csha=ip.csha
    and parent.curr=cs.csha
    and child.prev=cs.csha
    and cm.csha=child.curr
	and cp.application=parent.application
	and cp.csha=parent.prev

    and ip.csha in
    (
        select cs.csha
        from 
        (
            (select prev as csha from impact_pairs) 
            union 
            (select curr as csha from impact_pairs)
        )
        as ip
        ,commits as cs
        ,(
            (select application,csha from findbugs_summary_uni) 
            union 
            (select application,csha from pmd_uni) 
            union 
            (select application,csha from sonarqube_system_uni) 
        )
        as anal
        where cs.csha=anal.csha
        and cs.csha=ip.csha
        and cs.application=anal.application
    )
order by cs.application asc ,  cs.cwhen desc

'''

In [3]:
try:
    conn =  psycopg2.connect("dbname='' user='' password = '' host=''")
except:
    print ("I am unable to connect to the database")  

cur = conn.cursor()

cur.execute(sql_nc, ('%',))
pairs_nc = cur.fetchall()
cur.execute(sql_c, ('%',))
pairs_c = cur.fetchall()

In [4]:
COMP = 0
NON_COMP = 1

df_nc = pd.DataFrame(pairs_nc, columns = ['break_time', 'child_time', 'break_message', 'child_message',
                                          'break_csha', 'parent_csha', 'child_csha', 'break_email',
                                          'child_email', 'parent_time', 'parent_message'])
df_nc['compilable'] = NON_COMP
df_c = pd.DataFrame(pairs_c, columns = ['break_time', 'child_time', 'break_message', 'child_message',
                                          'break_csha', 'parent_csha', 'child_csha', 'break_email',
                                          'child_email', 'parent_time', 'parent_message'])
df_c['compilable'] = COMP

df = pd.concat([df_nc, df_c])
df.head()

,break_time,child_time,break_message,child_message,break_csha,parent_csha,child_csha,break_email,child_email,parent_time,parent_message,compilable
0,2016-05-17 15:20:36,2016-05-18 11:12:12,Wired up unsubscription for HystrixCollapsers ...,Typo\n,36bfd70,7b484eb,eeb751c,mjacobs@netflix.com,mjacobs@netflix.com,2016-05-16 13:58:56,Different fix for concurrency count command me...,1
1,2015-12-07 16:24:43,2015-12-09 09:57:27,Modelling command max-concurrency as a stream\n,Upgrade to RxJava 1.1.0\n,2ff16ff,3e8cb94,69151a2,mjacobs@netflix.com,mjacobs@netflix.com,2015-11-18 20:30:19,Bugfixes to accurately count only executions o...,1
2,2015-12-04 16:53:21,2015-11-03 09:08:09,Stop writing to HystrixRollingNumber for event...,Removed individual event writes from HystrixCo...,3c06a65,b4b9581,aaa9662,mjacobs@netflix.com,mjacobs@netflix.com,2015-10-22 14:56:37,Added object pooling to HystrixLatencyDistribu...,1
3,2015-12-04 16:48:16,2015-10-21 14:18:41,Wired HealthCounts up to HystrixCommandEventSt...,Decreasing memory allocations\n,fa4b7e2,745eeaa,86c37f1,mjacobs@netflix.com,mjacobs@netflix.com,2015-10-20 10:54:13,Reading from rolling percentile subject for la...,1
4,2015-11-18 20:30:19,2015-12-07 16:24:43,Bugfixes to accurately count only executions o...,Modelling command max-concurrency as a stream\n,3e8cb94,eae1f0a,2ff16ff,mjacobs@netflix.com,mjacobs@netflix.com,2015-11-18 10:37:43,Converted counters in HystrixThreadPoolMetrics...,1


### Feature Extraction
The following features will be extracted from raw data<br>
1) Message Similarity <br>
2) issue number (all of)<br>
3) time difference less than 1 hr?<br>
4) time messed up?<br>
5) same developer?<br>
6) Key word in commit message<br>

#### Feature 1 - Message Similarity

We make the assumption that the commit message of breaker and its child are similar because they describe the same issue.

In [5]:
N_GRAMS = 2
THRESHOLD = 0.8

def message_sim(row):
    distance = 1 - jaccard_distance(set(ngrams(row['break_message'],N_GRAMS)), set(ngrams(row['child_message'],N_GRAMS)))
    return distance

df['feature_1'] = df.apply(message_sim, axis = 1)

#### Feature 2 - Issue Number

The same idea as message similarity, but this feature capture only specific part of text message - issue number.

In [6]:
def issue_number(row):
    issue_number_break = re.findall('[A-Za-z]+[- ]\d+', row['break_message'])
    issue_number_child = re.findall('[A-Za-z]+[- ]\d+', row['child_message'])

    if len(issue_number_break) == 0 or len(issue_number_child) == 0:
        return -1
    elif issue_number_break == issue_number_child:
        return 1
    else:
        return -1

df['feature_2'] = df.apply(issue_number, axis = 1)

#### Feature 3 - Time Difference > 1 hour

This feature capture the behavior that the developers break the compilability of the software, they will fix it immediately

In [7]:
HOUR = 3600
DAY = HOUR * 24

df['feature_3'] = df.apply(lambda x: 1 if (x['child_time'] - x['break_time']).total_seconds() > HOUR else -1, axis =1)


#### Feature 4 - whether time messed up?

Some time we observe that time log in the repository are messed up. This can be clue that something wrong happens.

In [8]:
df['feature_4'] = df.apply(lambda x: 1 if (x['child_time'] - x['break_time']).total_seconds() > 0 else -1, axis =1)

#### Feature 5 - whether same developer?

We assume that the deveopers who break the compilability and fix it will be the same. It turns out that this feature does not contribute much to the prediction so we don't use it.

In [9]:
# df['feature_5'] = df['break_email'] == df['child_email']

#### Feature 6 - whether contain key words? 

Frome feature engineering, we found some keywords that appear in the message of commit just after the broken ones. We will use this as a feature. We can use the full list of keywords, but it turns out that only issue-like words and 'oops' are enough.

In [10]:
def contain_kw(row):
    issue_number_break = re.findall('[A-Za-z]+[- ]\d+', row['break_message'])
    if issue_number_break or 'oops' in row['child_message'].lower() : return 1
    else: return -1
    
df['feature_6'] = df.apply(contain_kw, axis = 1)




#### Feature 7 - Log Time Difference 

This feature capture the same behavior as feature 3 but in continuous domain.
Note: We also try time difference between parents and child to capture the behavior that the if developer commits to early, they will carelessly broke the compilability. It turns out that comparing breaker commits and child is better.


In [11]:
def log_(row):
    hour = (row['child_time'] - row['break_time']).total_seconds()/3600.0
    if hour >= 1.0/3600:
        return np.log(hour)
    else: return np.log(0.01/3600)
    
df['feature_7'] = df.apply(log_, axis =1)

#### Let's see all these features

In [12]:
df.head(5)

,break_time,child_time,break_message,child_message,break_csha,parent_csha,child_csha,break_email,child_email,parent_time,parent_message,compilable,feature_1,feature_2,feature_3,feature_4,feature_6,feature_7
0,2016-05-17 15:20:36,2016-05-18 11:12:12,Wired up unsubscription for HystrixCollapsers ...,Typo\n,36bfd70,7b484eb,eeb751c,mjacobs@netflix.com,mjacobs@netflix.com,2016-05-16 13:58:56,Different fix for concurrency count command me...,1,0.000000,-1,1,1,-1,2.988708
1,2015-12-07 16:24:43,2015-12-09 09:57:27,Modelling command max-concurrency as a stream\n,Upgrade to RxJava 1.1.0\n,2ff16ff,3e8cb94,69151a2,mjacobs@netflix.com,mjacobs@netflix.com,2015-11-18 20:30:19,Bugfixes to accurately count only executions o...,1,0.033333,-1,1,1,-1,3.726791
2,2015-12-04 16:53:21,2015-11-03 09:08:09,Stop writing to HystrixRollingNumber for event...,Removed individual event writes from HystrixCo...,3c06a65,b4b9581,aaa9662,mjacobs@netflix.com,mjacobs@netflix.com,2015-10-22 14:56:37,Added object pooling to HystrixLatencyDistribu...,1,0.271739,-1,-1,-1,-1,-12.793859
3,2015-12-04 16:48:16,2015-10-21 14:18:41,Wired HealthCounts up to HystrixCommandEventSt...,Decreasing memory allocations\n,fa4b7e2,745eeaa,86c37f1,mjacobs@netflix.com,mjacobs@netflix.com,2015-10-20 10:54:13,Reading from rolling percentile subject for la...,1,0.090909,-1,-1,-1,-1,-12.793859
4,2015-11-18 20:30:19,2015-12-07 16:24:43,Bugfixes to accurately count only executions o...,Modelling command max-concurrency as a stream\n,3e8cb94,eae1f0a,2ff16ff,mjacobs@netflix.com,mjacobs@netflix.com,2015-11-18 10:37:43,Converted counters in HystrixThreadPoolMetrics...,1,0.178571,-1,1,1,-1,6.113476


### Train the Model With Logistic Regression

In order to make class balance, we use only 1200 of total commits from the repository to develop the model. The compilable and uncompilable instances are roughly equal. We randomly separate those data to train_set (900 instances) and test set (300 instance).

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,confusion_matrix

In [14]:


X = df[['feature_1', 'feature_2', 'feature_3', 'feature_4','feature_6','feature_7']].iloc[0:1200]
y = df[['compilable']].iloc[0:1200]
X_train, X_test, y_train, y_test = train_test_split(X,y['compilable'], random_state = 0)

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)
y_pre_proba = model.predict_proba(X_test)
y_pre = model.predict(X_test)




### Model Evaluation

We use predict the class from test data and use various of evaluation metrices.

In [15]:
print('AUC: {:.4f}'.format(roc_auc_score(y_test, y_pre_proba[:,1])))
print('F1: {:.4f}'.format(f1_score(y_test, y_pre)))
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pre))
print('Model Coefficient')
print(model.coef_[0])


AUC: 0.8540
F1: 0.7686
Confusion Matrix
[[143  19]
 [ 40  98]]
Model Coefficient
[ 1.91358584  1.1860817  -0.17860799  1.31142585  0.74077677 -0.18760859]


### Cross-Validation K-Fold
Using 10 folds to cross-validate the results

In [16]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [17]:
kf = KFold(n_splits=10)
kfold = KFold(n_splits=10)

model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, np.array(y).ravel(), cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))


Accuracy: 69.083% (23.524%)


In order to make sure that F1 score is good, we use dummy classifer to create the baseline accuracy.

In [18]:
from sklearn.dummy import DummyClassifier
X_train, X_test, y_train, y_test = train_test_split(X,y['compilable'], random_state = 0)

dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
print("Random Classifier Score: %.2E"%dummy_majority.score(X_test, y_test))

Random Classifier Score: 4.60E-01
